In [12]:
import pandas as pd
from statsmodels.stats.multicomp import (pairwise_tukeyhsd,MultiComparison)

In [27]:
rpe_file = '~/Desktop/rpe.csv'
rpe_data = pd.read_csv(rpe_file)
rpe_data.shape


(8860, 14)

In [28]:
print(list(rpe_data.columns))

['Date', 'PlayerID', 'Training', 'SessionType', 'Duration', 'RPE', 'SessionLoad', 'DailyLoad', 'AcuteLoad', 'ChronicLoad', 'AcuteChronicRatio', 'ObjectiveRating', 'FocusRating', 'BestOutOfMyself']


In [30]:
rpe_data = rpe_data.dropna(subset=['RPE'])
rpe_data.shape

(7621, 14)

In [7]:
rpe_data.head()

,Date,PlayerID,Training,SessionType,Duration,RPE,SessionLoad,DailyLoad,AcuteLoad,ChronicLoad,AcuteChronicRatio,ObjectiveRating,FocusRating,BestOutOfMyself
0,2018-07-21,13,Yes,Mobility/Recovery,5.0,0.0,0.0,446.0,323.1,303.25,1.07,NaN,NaN,Not at all
1,2018-07-21,13,Yes,Game,29.0,7.0,203.0,NaN,NaN,NaN,NaN,NaN,NaN,Not at all
2,2018-07-21,13,Yes,Game,27.0,9.0,243.0,NaN,NaN,NaN,NaN,NaN,NaN,Not at all
3,2018-07-20,11,Yes,Game,36.0,6.0,216.0,534.0,256.3,344.18,0.74,9.0,10.0,Absolutely
4,2018-07-20,11,Yes,Game,36.0,8.0,288.0,NaN,NaN,NaN,NaN,9.0,10.0,Absolutely


In [19]:
rpe_data.groupby(rpe_data["SessionType"])["SessionType"].count()


SessionType
Combat                193
Conditioning          960
Game                  463
Mobility/Recovery     992
Skills               1859
Speed                1245
Strength             1909
Name: SessionType, dtype: int64

In [31]:
rpe_data["RPE"].count()


7621

In [33]:
pairwise_tukeyhsd(rpe_data["RPE"],rpe_data["SessionType"]).summary()

group1,group2,meandiff,p-adj,lower,upper,reject
Combat,Conditioning,2.1053,0.001,1.7308,2.4799,True
Combat,Game,2.0734,0.001,1.6666,2.4803,True
Combat,Mobility/Recovery,-3.1398,0.001,-3.5134,-2.7663,True
Combat,Skills,-0.1147,0.9,-0.4737,0.2444,False
Combat,Speed,-0.1785,0.7567,-0.5458,0.1888,False
Combat,Strength,0.8609,0.001,0.5023,1.2196,True
Conditioning,Game,-0.0319,0.9,-0.3006,0.2368,False
Conditioning,Mobility/Recovery,-5.2452,0.001,-5.4601,-5.0302,True
Conditioning,Skills,-2.22,0.001,-2.4087,-2.0313,True
Conditioning,Speed,-2.2839,0.001,-2.4878,-2.0799,True


Since most of groups have different mean for RPE, we can include that SessionType is highly correlated to RPE